## Sparse Grid Regression with the Combination Technique: 
This notebook provides an overview on how to use the Regression with the (spatially adaptve) Combination Technique. The complexity of the computation is lowered while still providing accurate results compared to full grids. The class $\texttt{Regression}$ implements those functionalities. Its methods $\texttt{train}$, $\texttt{optimize\_coefficients}$ and $\texttt{test}$ encapsulate the most important steps.

First, an object of class $\texttt{Regression}$ has to be initialized. The data set and its target values have to be given in the constructor. Additionally, a regularization parameter $\lambda$ and the choice for the regularization matrix (C or I) have to be determined. Additionally, the range to which the data is scaled to can be changed. The standard value is $[0.05,0.95]$.

In the next step, the regression object can be trained. The method $\texttt{train}$ takes 4 parameters. The percentage of testdata decides, how the data set given in the constructor is split (training, validation and test). Minimum and maximum level have to be given for the Combination Technique and with the last boolean parameter, white noise can be added to the target values of the training and validation data. The $\texttt{train}$ method returns a combiObject. With this, predictions of new data points can be made.

As a next, optional step, the combination technique can be optimized (Opticom). Therefore, the method $\texttt{optimize\_coefficients}$ of Regression has to be called. It needs the combiObject from the previous method and the choice of option: 1 -> Garcke approach, 2 -> least squares based, 3 -> error based approach.

The last step is to test the computed function with the help of $\texttt{test}$. It again needs the combiObject and it computes the mean squared error of the test data. 

### Regression with the normal combination technique:

In [ ]:
# import sparseSpACE
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sparseSpACE.DEMachineLearning import *
import math

def construct_dataset(dim, function, num_points):
    grid_points = np.random.rand(num_points,dim)
    y_vals = np.array([function(x) for x in grid_points])
    return grid_points, y_vals.flatten()

function = GenzGaussian((0.5,0.5), (10,10))
data, target = construct_dataset(2, function, 300)

operation = Regression(data, target, 0, 'C')


print("Plot of the data samples: ")
operation.plot_dataset()
print("Plot of the function: ")
function.plot((0,0),(1,1))
combiObject = operation.train(0.2, 1, 4, noisy_data=False)
combiObject.print_resulting_combi_scheme(operation=operation)
print("Sparse Grid:")
combiObject.print_resulting_sparsegrid(markersize=20)
print("Plot of Regression:")
combiObject.plot(contour=True)
print("Testing error: ")
print(operation.test(combiObject))
print("Time used: ")
print(combiObject.get_time_used())


### Regression with the spatially adaptive combination technique:

The same steps as above can be performed. For the training methods, the following parameters can be specified: 
- percentage of test data: how many data points are used for testing
- margin: at what points will be refined (at those with error >= margin*max_error 
- tolerance: stopping criterion depending on error
- max_evaluations: refinement stops when current evaluations >= max_evalauations
- do_plot: specifiy whether each refinement step is plotted
- noisy_data: decide whether white noise is added to the training target values

In [ ]:
# import sparseSpACE
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sparseSpACE.DEMachineLearning import *
import math

def construct_dataset(dim, function, num_points):
    grid_points = np.random.rand(num_points,dim)
    y_vals = np.array([function(x) for x in grid_points])
    return grid_points, y_vals.flatten()

function = GenzGaussian((0.7,0.7), (30,30))
data, target = construct_dataset(2, function, 300)

operation = Regression(data, target, 0, 'C')


print("Plot of the data samples: ")
operation.plot_dataset()
print("Plot of the function: ")
function.plot((0,0),(1,1))
adaptiveCombiInstanceSingleDim = operation.train_spatially_adaptive(0.2, 0.7, 10.**-5, 18, do_plot=False, noisy_data=False)
adaptiveCombiInstanceSingleDim.print_resulting_combi_scheme(operation=operation)
print("Sparse Grid:")
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid(markersize=20)
print("Plot of Regression:")
adaptiveCombiInstanceSingleDim.plot(contour=True)
print("Testing error: ")
print(operation.test_spatially_adaptive(adaptiveCombiInstanceSingleDim))